In [1]:
import tensorflow as tf
from tensorflow.python.keras.datasets import cifar10
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import layers, activations, optimizers, models,losses
import os
import matplotlib.pyplot as plt

c:\programdata\anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Model Parameters

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), "saved_models")
model_name = 'keras_cifar10_trained_model.h5'

# Load data into memory

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# Define Model

In [4]:
cifar_model = models.Sequential()
cifar_model.add(layers.Conv2D(filters=32, kernel_size=[2, 2], strides=2, activation=activations.relu))
cifar_model.add(layers.Conv2D(filters=32, kernel_size=[2, 2], strides=2, activation=activations.relu))
cifar_model.add(layers.MaxPool2D(pool_size=(2, 2)))
cifar_model.add(layers.Dropout(0.25))

cifar_model.add(layers.Conv2D(filters=64, kernel_size=[2, 2], strides=2, activation=activations.relu))
cifar_model.add(layers.Conv2D(filters=64, kernel_size=[2, 2], strides=2, activation=activations.relu))
cifar_model.add(layers.MaxPool2D(pool_size=(2, 2)))
cifar_model.add(layers.Dropout(0.25))

cifar_model.add(layers.Flatten())
cifar_model.add(layers.Dense(units=512, activation=activations.relu))
cifar_model.add(layers.Dropout(0.25))
cifar_model.add(layers.Dense(units=num_classes, activation=activations.softmax))

cifar_model.compile(
    optimizer=optimizers.RMSprop(lr=0.001, decay=1e-6),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy']
)


# Run the model

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    cifar_model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

else:
    print("Using real time data augmentation")
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    #Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


# Accuracy Score

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])